## Pitch Manipulation

In [1]:
!pip install praat.parselmouth

     |████████████████████████████████| 10.7 MB 14.3 MB/s 


In [2]:
import parselmouth
sound = parselmouth.Sound("newrec.wav")

In [3]:
from IPython.display import Audio
Audio(data=sound.values, rate=sound.sampling_frequency)

In [4]:
from parselmouth.praat import call

manipulation = call(sound, "To Manipulation", 0.01, 75, 600)

In [5]:
type(manipulation)

parselmouth.Data

In [6]:
pitch_tier = call(manipulation, "Extract pitch tier")

call(pitch_tier, "Multiply frequencies", sound.xmin, sound.xmax, 1)

call([pitch_tier, manipulation], "Replace pitch tier")
sound_octave_up = call(manipulation, "Get resynthesis (overlap-add)")
type(sound_octave_up)

parselmouth.Sound

In [7]:
Audio(data=sound_octave_up.values, rate=sound_octave_up.sampling_frequency)

In [8]:
sound_octave_up.save("newrec.wav", "WAV")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: PraatWarning: Writing samples to audio file: 1 out of 261954 samples have been clipped.
Advice: you could scale the amplitudes or write to a binary file.
  """Entry point for launching an IPython kernel.


In [9]:
Audio(filename="newrec.wav")

In [10]:
def change_pitch(sound, semitone):
    print(semitone)
    print(pow(2,semitone/12)) 
    manipulation = call(sound, "To Manipulation", 0.01, 75, 600)

    pitch_tier = call(manipulation, "Extract pitch tier")

    call(pitch_tier, "Multiply frequencies", sound.xmin, sound.xmax, pow(2,semitone/12))

    call([pitch_tier, manipulation], "Replace pitch tier")
    return call(manipulation, "Get resynthesis (overlap-add)")

In [12]:
import ipywidgets
import glob

def interactive_change_pitch(audio_file, semitone):
    
    sound = parselmouth.Sound(audio_file)
    sound_changed_pitch = change_pitch(sound, semitone)
    return Audio(data=sound_changed_pitch.values, rate=sound_changed_pitch.sampling_frequency)
maxi=input("Enter your maximum possible pitch: ")
print(maxi)
mini=input("Enter your minimum possible pitch:")
print(mini)
w = ipywidgets.interact(interactive_change_pitch,
                        audio_file=ipywidgets.Dropdown(options=sorted(glob.glob("*.wav")), value="newrec.wav"),
                        semitone=ipywidgets.FloatSlider(min=mini, max=maxi, step=1, value=0))
                        

Enter your maximum possible pitch: 12
12
Enter your minimum possible pitch:-12
-12


interactive(children=(Dropdown(description='audio_file', options=('newrec.wav',), value='newrec.wav'), FloatSl…

## Data Generation

In [13]:
import numpy as np

In [14]:
snd = parselmouth.Sound("newrec.wav")

In [15]:
pitch = snd.to_pitch() 

In [16]:
def similar_pitch(pitch,n):
    pitch_values = pitch.selected_array['frequency']
    pitch_values[pitch_values==0] = np.nan
    j=0
    l=input("Enter the threshold value(frame): ")
    #st=input("Enter the number of semitones by which audio is changed: ")
    k=1
    for i in range (0,len(pitch_values)):
      if(pitch_values[i]!='nan'):
        val=pitch_values[i]
        k=0;
        similar_pitch_set=[ ]
        for j in range(i,len(pitch_values)):
          if(pitch_values[j]!='nan'):
             if (float(pitch_values[j])-float(val) <= (float(val)*pow(2,n/24))-float(val)):
                similar_pitch_set.append(pitch_values[j])
                k=k+1
             if (float(pitch_values[j])-float(val) > (float(val)*pow(2,n/24))-float(val)):
                break
          else:
             break
        i=i+k
        if(k>=int(l)):
          print(similar_pitch_set)


In [17]:
n=int(input("Enter the number of semitones by which the pitch is changed: "))
similar_pitch(pitch,n)

Enter the number of semitones by which the pitch is changed: 2
Enter the threshold value(frame): 4
[129.06741665316153, 128.8664784059826, 127.09781901485897, 127.45002561852586, 128.0654730575122, 128.85682959430576, 130.57087256293207, 132.4252057810676, 133.2736853343604, 131.7391750508383, 130.33623888070304, 129.55761532055405, 128.06644897848415, 127.80708427613574, 128.07190341781464, 128.39964936398428, 128.73981426414682, 129.1667961540863, 129.7689968907243, 130.16645486182253, 129.6814490102743, 128.2841618769382, 125.60801977367115, 124.48805141089437, 79.31350024758551, 127.60805109489327, 129.29700008395304, 130.14001245795512, 130.48087156258347, 130.50832683154132, 130.3652963786081, 129.80615619526634, 129.5727665337657, 129.34125714410197, 128.81922062443408, 127.91115473595465, 123.96692675785478, 125.44104701221055, 128.93062019312117, 129.40647379572854, 129.0088624046709, 129.14948926172897, 129.18726747005854, 129.770985809939, 129.19657582074936, 129.20190502322

## Understanding Pitch Manipulation

In [18]:
def change_pitch(pitch,n):
    # Extract selected pitch contour, and
    # replace unvoiced samples by NaN to not plot
    pitch_values = pitch.selected_array['frequency']
    pitch_values[pitch_values==0] = np.nan
    print(pitch_values)
    for i in range(0,len(pitch_values)):
      pitch_values[i]=pitch_values[i]*pow(2,n/12)
    print(pitch_values)

In [19]:
m = int(input("Number of semitones by which pitch is changed: "))


Number of semitones by which pitch is changed: 2


In [20]:
change_pitch(pitch,m)

[         nan          nan          nan          nan          nan
          nan          nan          nan          nan          nan
          nan          nan          nan          nan          nan
          nan          nan          nan          nan          nan
          nan          nan          nan          nan          nan
          nan          nan          nan          nan          nan
          nan          nan          nan          nan          nan
          nan          nan          nan          nan          nan
          nan          nan          nan          nan          nan
          nan          nan          nan          nan          nan
 129.06741665 128.86647841 127.09781901 127.45002562 128.06547306
 128.85682959 130.57087256 132.42520578 133.27368533 131.73917505
 130.33623888 129.55761532 128.06644898 127.80708428 128.07190342
 128.39964936 128.73981426 129.16679615 129.76899689 130.16645486
 129.68144901 128.28416188 125.60801977 124.48805141  79.31350025
          